In [1]:
## Client Code (Updated for File Sharing)

import socket
import threading
import tkinter as tk
from tkinter import scrolledtext, simpledialog
import os
import webbrowser

class ChatClient:
    def __init__(self, host, port=55555):
        self.client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client.connect((host, port))

        self.root = tk.Tk()
        self.root.title("Chat Application")
        self.root.configure(bg='#282C34')

        self.chat_area = scrolledtext.ScrolledText(
            self.root, wrap=tk.WORD, state='disabled',
            bg='#1E1E1E', fg='#00FF00', font=('Consolas', 12), padx=10, pady=10
        )
        self.chat_area.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        self.message_frame = tk.Frame(self.root, bg='#282C34')
        self.message_frame.pack(padx=10, pady=5, fill=tk.X)

        self.input_area = tk.Entry(
            self.message_frame, font=('Consolas', 12),
            bg='#3B3F46', fg='#FFFFFF', insertbackground='white', width=80
        )
        self.input_area.grid(row=0, column=0, padx=5, pady=5, sticky='we')
        self.input_area.bind("<Return>", self.send_message)

        self.send_button = tk.Button(
            self.message_frame, text="Send", command=self.send_message,
            bg='#61AFEF', fg='white', font=('Consolas', 10, 'bold'), width=10
        )
        self.send_button.grid(row=0, column=1, padx=5, pady=5)

        self.message_frame.columnconfigure(0, weight=1)

        self.nickname = self.prompt_nickname()
        if self.nickname:
            self.client.send(self.nickname.encode('ascii'))
        else:
            self.root.destroy()

        threading.Thread(target=self.receive_messages, daemon=True).start()

        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        self.root.mainloop()

    def prompt_nickname(self):
        return simpledialog.askstring("Nickname", "Choose a nickname:", parent=self.root)

    def receive_messages(self):
        while True:
            try:
                message = self.client.recv(1024).decode('ascii')
                if message.startswith("FILE|"):
                    _, filename, filesize = message.split('|')
                    filesize = int(filesize)

                    file_data = b''
                    while len(file_data) < filesize:
                        file_data += self.client.recv(1024)

                    file_path = os.path.join(os.getcwd(), filename)
                    with open(file_path, 'wb') as file:
                        file.write(file_data)

                    self.display_message(f"Received file: {filename}")
                    open_button = tk.Button(
                        self.chat_area, text=f"Open {filename}", command=lambda: webbrowser.open(file_path),
                        bg='#98C379', fg='black', font=('Consolas', 10)
                    )
                    self.chat_area.window_create(tk.END, window=open_button)
                    self.chat_area.insert(tk.END, "\n")
                else:
                    self.display_message(message)
            except Exception as e:
                print(f"Error: {e}")
                self.client.close()
                break

    def send_message(self, event=None):
        message = self.input_area.get().strip()
        if message:
            self.client.send(message.encode('ascii'))
            self.input_area.delete(0, tk.END)

    def display_message(self, message):
        self.chat_area.configure(state='normal')
        self.chat_area.insert(tk.END, f"{message}\n")
        self.chat_area.configure(state='disabled')
        self.chat_area.yview(tk.END)

    def on_close(self):
        self.client.close()
        self.root.destroy()

if __name__ == "__main__":
    server_ip = input("Enter the server IP address: ")
    ChatClient(server_ip)


Error: [WinError 10053] An established connection was aborted by the software in your host machine
